In [83]:
# TensorFlow and TF-Hub modules.
from absl import logging

import tensorflow as tf
import tensorflow_hub as hub
# from tensorflow_docs.vis import embed

logging.set_verbosity(logging.ERROR)

# Some modules to help with reading the UCF101 dataset.
import random
import re
import os
import tempfile
import ssl
import cv2
import numpy as np
import h5py
import datetime as dt

# Some modules to display an animation using imageio.
import imageio
from IPython import display

from urllib import request  # requires python3

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import keras
from keras.src.applications.xception import Xception
from keras.applications.xception import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.utils import to_categorical

In [84]:
num_classes = 12

# Augmentations
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [85]:
# Utilities to fetch videos from UCF101 dataset
UCF_ROOT = "https://www.crcv.ucf.edu/THUMOS14/UCF101/UCF101/"
_VIDEO_LIST = None
_CACHE_DIR = tempfile.mkdtemp()
# As of July 2020, crcv.ucf.edu doesn't use a certificate accepted by the
# default Colab environment anymore.
unverified_context = ssl._create_unverified_context()


def list_ucf_videos():
  """Lists videos available in UCF101 dataset."""
  global _VIDEO_LIST
  if not _VIDEO_LIST:
    index = request.urlopen(UCF_ROOT, context=unverified_context).read().decode("utf-8")
    videos = re.findall("(v_[\w_]+\.avi)", index)
    _VIDEO_LIST = sorted(set(videos))
  return list(_VIDEO_LIST)


def fetch_ucf_video(video):
  """Fetches a video and cache into local filesystem."""
  cache_path = os.path.join(_CACHE_DIR, video)
  if not os.path.exists(cache_path):
    urlpath = request.urljoin(UCF_ROOT, video)
    print("Fetching %s => %s" % (urlpath, cache_path))
    data = request.urlopen(urlpath, context=unverified_context).read()
    open(cache_path, "wb").write(data)
  return cache_path


# Utilities to open video files using CV2
def crop_center_square(frame):
  y, x = frame.shape[0:2]
  min_dim = min(y, x)
  start_x = (x // 2) - (min_dim // 2)
  start_y = (y // 2) - (min_dim // 2)
  return frame[start_y:start_y+min_dim,start_x:start_x+min_dim]


def data_generator(paths, labels, batch_size=16):
    while True:
        for i in range(0, len(paths), batch_size):
            batch_paths = paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            
            # If last batch is smaller than batch_size
            if len(batch_paths) < batch_size:
                pad_size = batch_size - len(batch_paths)
                batch_paths += batch_paths[:pad_size]
                batch_labels += batch_labels[:pad_size]
            
            batch_data = []
            for path in batch_paths:
                frames = load_video(path)
                if frames is not None:
                    batch_data.append(frames)
            
            batch_data = np.array(batch_data) 
            yield batch_data, np.array(batch_labels)


def load_video(path, max_frames=10, resize=(224, 224)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]

            frame = datagen.random_transform(frame)

            frames.append(frame)

            if len(frames) == max_frames:
                break

        cap.release()

        # Ensure exactly max_frames are returned
        if len(frames) < max_frames:
            frames += [np.zeros((resize[0], resize[1], 3))] * (max_frames - len(frames))
        
    except Exception as e:
        print(e)
        frames = [np.zeros((resize[0], resize[1], 3))] * max_frames

    return np.array(frames) / 255.0



def to_gif(images):
  converted_images = np.clip(images * 255, 0, 255).astype(np.uint8)
  gif = imageio.mimsave('./animation.gif', converted_images, duration=40)
  return gif


# List files and ignore .DS_Store if on a Mac
def list_files(directory):
    visible_files = []
    for file in os.listdir(directory):
        if not file.startswith('.'):
            visible_files.append(file)

    return visible_files


def video_to_frames(video_path, img_size=(64, 64), sequence_length=30):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # frame = cv2.resize(frame, img_size)
        img = keras.utils.load_img(frame, target_size=(224, 224))
        x = keras.utils.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        frames.append(x)
        if len(frames) == sequence_length:
            break
    cap.release()

    if len(frames) < sequence_length:
        return None  # Ignore short videos

    return np.array(frames)


# Get paths and labels
def load_dataset(folder_path, print_path=False):
    # classes = os.listdir(folder_path)
    classes = list_files(folder_path)
    paths, true_labels, activities = [], [], []

    for label, activity in enumerate(classes):
        activity_folder = os.path.join(folder_path, activity)
        files = list_files(activity_folder)
        num_files = len(files)
        current = 1
        for video_file in files:
            video_path = os.path.join(activity_folder, video_file)
            paths.append(video_path)
            true_labels.append(label)
            activities.append(activity)

            if print_path:
                print(f'{current}/{num_files}\t{video_path}')

            current += 1

    return paths, true_labels, activities


class I3DModelLayer(tf.keras.layers.Layer):
    def __init__(self, i3d_model, **kwargs):
        super(I3DModelLayer, self).__init__(**kwargs)
        self.i3d_model = i3d_model

    def call(self, inputs):
        output = self.i3d_model(rgb_input=inputs)['default']
        return output

    def get_config(self):  # Enables model.save()
        config = super(I3DModelLayer, self).get_config()
        config.update({"i3d_model": None})  # Placeholder, i3d_model is not serializable
        return config
    

def built_model(print_details=True):
    # Load i3d-kinetics-400 model
    i3d = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

    class I3DModelLayer(tf.keras.layers.Layer):
        def __init__(self, i3d_model, **kwargs):
            super(I3DModelLayer, self).__init__(**kwargs)
            self.i3d_model = i3d_model

        def call(self, inputs):
            output = self.i3d_model(rgb_input=inputs)['default']
            return output

        def get_config(self):  # Enables model.save()
            config = super(I3DModelLayer, self).get_config()
            config.update({"i3d_model": None})  # Placeholder, i3d_model is not serializable
            return config
        
    # Freeze layers
    # i3d.trainable = False
    i3d_layer = I3DModelLayer(i3d)

    for var in i3d_layer.variables:
        var.trainable = False

    inputs = tf.keras.Input(shape=(10, 224, 224, 3), name="input_layer")
    i3d_layer = I3DModelLayer(i3d, name="i3d_model_layer")
    x = i3d_layer(inputs)
    x = tf.keras.layers.Dropout(0.2, name="dropout")(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax', name="dense")(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001, clipvalue=1.0),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    )

    if print_details:
        model.summary()

    return model


# Get paths and labels
def load_dataset(folder_path, print_path=False):
    # classes = os.listdir(folder_path)
    classes = list_files(folder_path)
    paths, true_labels, activities = [], [], []

    for label, activity in enumerate(classes):
        activity_folder = os.path.join(folder_path, activity)
        files = list_files(activity_folder)
        num_files = len(files)
        current = 1
        for video_file in files:
            video_path = os.path.join(activity_folder, video_file)
            paths.append(video_path)
            true_labels.append(label)
            activities.append(activity)

            if print_path:
                print(f'{current}/{num_files}\t{video_path}')

            current += 1

    return paths, true_labels, activities


def make_dataset(paths, labels):
    all_data = []
    all_labels = []

    for p in enumerate(paths): 
        print(f'{p[0] + 1}/{len(paths)}\t{p[1]}')
        frames = load_video(p[1])
        all_data.append(frames)
        lab = labels[p[0]]
        all_labels.append(lab)

    return all_data, all_labels


Load dataset

In [7]:
train_dir = f"../../downloads/fr_10s/train_fr_10s"
train_paths, train_labels, train_activities = load_dataset(train_dir)

In [8]:
X_train, y_train = make_dataset(train_paths, train_labels)

1/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393437904103853201_s_1.mp4
2/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393439141054434449_s_1.mp4
3/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393443393072057489_s_1.mp4
4/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393443504741207185_s_1.mp4
5/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393443844043623569_s_1.mp4
6/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393443844043623569_s_2.mp4
7/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393444200525909137_s_1.mp4
8/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393444432454143121_s_1.mp4
9/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393444599957867665_s_1.mp4
10/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying to sleep\7393446128966225041_s_1.mp4
11/1136	../../downloads/fr_10s/train_fr_10s\Asleep-Trying t

In [69]:
# Convert to categorical
y_train = to_categorical(y_train, num_classes=num_classes)

# Train, test, split
train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_paths, train_labels, test_size=0.2, random_state=42
)

train_gen = data_generator(train_paths, train_labels, batch_size=16)
val_gen = data_generator(val_paths, val_labels, batch_size=16)

In [70]:
sample_batch, sample_labels = next(train_gen)
print(f"Batch shape: {sample_batch.shape}")  # Expected: (16, 10, 224, 224, 3)
print(f"Labels shape: {sample_labels.shape}")  # Expected: (16,)

Batch shape: (16, 10, 224, 224, 3)
Labels shape: (16,)


Train model

In [74]:
# Callbacks
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 0.001)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.5, 
    patience=4, 
    min_lr=1e-6, 
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_sparse_categorical_accuracy', 
    patience=5,
    mode='max',
    restore_best_weights=True
)

# Build model
model = built_model(print_details=False)

In [75]:
history = model.fit(
    train_gen,
    steps_per_epoch=len(train_paths) // 16,
    validation_data=val_gen,
    validation_steps=len(val_paths) // 16,
    epochs=10,
    callbacks=[early_stopping, lr_schedule]
)

Epoch 1/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 157s 3s/step - loss: 4.0784 - sparse_categorical_accuracy: 0.1607 - val_loss: 2.4317 - val_sparse_categorical_accuracy: 0.2557 - learning_rate: 0.0010
Epoch 2/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 150s 3s/step - loss: 3.0433 - sparse_categorical_accuracy: 0.2084 - val_loss: 2.0865 - val_sparse_categorical_accuracy: 0.3256 - learning_rate: 0.0010
Epoch 3/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 144s 3s/step - loss: 2.8888 - sparse_categorical_accuracy: 0.2475 - val_loss: 1.9148 - val_sparse_categorical_accuracy: 0.3605 - learning_rate: 0.0010
Epoch 4/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 128s 3s/step - loss: 2.4888 - sparse_categorical_accuracy: 0.2770 - val_loss: 1.7608 - val_sparse_categorical_accuracy: 0.4128 - learning_rate: 0.0010
Epoch 5/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - loss: 2.5638 - sparse_categorical_accuracy: 0.2748 - val_loss: 1.6139 - val_sparse_categorical_accuracy: 0.4360 - learning_rate: 0.0010
Epoch 6/10
45/45 ━━━━━━━━━━━━━━━━━━━━ 136s 3s/step - lo

In [79]:
date_time_format = '%Y-%m-%d-%H-%M-%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

model_name = f'./models/{current_date_time_string}-i3d-transfer-model.keras'

# Save model
model.save(model_name)
print(f"Saved {model_name}")

# Save weights
model_weights = f'./weights/{current_date_time_string}-i3d-transfer.weights.h5'
model.save_weights(model_weights)

print(f"Saved {model_weights}")

Saved ./models/2024-11-21-03-45-07-i3d-transfer-model.keras
Saved ./weights/2024-11-21-03-45-07-i3d-transfer.weights.h5


## Test

In [93]:
class I3DModelLayer(tf.keras.layers.Layer):
    def __init__(self, i3d_model, **kwargs):
        super(I3DModelLayer, self).__init__(**kwargs)
        self.i3d_model = i3d_model

    def call(self, inputs):
        # Ensure inputs are passed correctly to the I3D model
        output = self.i3d_model(inputs)  # Pass inputs directly if signature doesn't use 'rgb_input'
        return output

    def compute_output_shape(self, input_shape):
        # You need to specify the output shape explicitly
        # Assuming the output is a 400-dimensional vector (e.g., from the I3D model)
        return (input_shape[0], 400)  # Modify this depending on the exact output of your I3D model

    def get_config(self):  # Enables model.save()
        config = super(I3DModelLayer, self).get_config()
        config.update({"i3d_model": None})  # Placeholder, i3d_model is not serializable
        return config



In [89]:
model_path = './models/2024-11-21-03-45-07-i3d-transfer-model.keras'

model = tf.keras.models.load_model(
    model_path, 
    compile=False, 
    custom_objects={"I3DModelLayer": I3DModelLayer}
)

TypeError: Could not deserialize class 'Functional' because its parent module keras.src.models.functional cannot be imported. Full object config: {'module': 'keras.src.models.functional', 'class_name': 'Functional', 'config': {'name': 'functional_19', 'trainable': True, 'layers': [{'module': 'keras.layers', 'class_name': 'InputLayer', 'config': {'batch_shape': [None, 10, 224, 224, 3], 'dtype': 'float32', 'sparse': False, 'name': 'input_layer'}, 'registered_name': None, 'name': 'input_layer', 'inbound_nodes': []}, {'module': None, 'class_name': 'I3DModelLayer', 'config': {'name': 'i3d_model_layer', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'i3d_model': None}, 'registered_name': 'I3DModelLayer', 'build_config': {'input_shape': [None, 10, 224, 224, 3]}, 'name': 'i3d_model_layer', 'inbound_nodes': [{'args': [{'class_name': '__keras_tensor__', 'config': {'shape': [None, 10, 224, 224, 3], 'dtype': 'fl

In [ ]:
# If loading weights
num_classes = 12 

# Rebuild model
model_reloaded = built_model(print_details=False)

# Load weights
weights = './weights/2024-11-21-03-45-07-i3d-transfer.weights.h5'
model_reloaded.load_weights(weights, skip_mismatch= True, by_name = True)

print("Weights loaded")

In [96]:
test_dir = f"../../downloads/fr_10s/test_fr_10s"
test_paths, test_labels, test_activities = load_dataset(test_dir)

In [97]:
X_test, y_test = make_dataset(test_paths, test_labels)
y_test = to_categorical(y_test, num_classes=num_classes)

1/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394376363047963793_s_5.mp4
2/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7393498639236385937_s_10.mp4
3/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394377634358283409_s_2.mp4
4/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394377634358283409_s_3.mp4
5/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7393498639236385937_s_11.mp4
6/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394376363047963793_s_4.mp4
7/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394376363047963793_s_6.mp4
8/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394377634358283409_s_1.mp4
9/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394377110372273297_s_10.mp4
10/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394376070990187665_s_1.mp4
11/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394376363047963793_s_3.mp4
12/1106	../../downloads/fr_10s/test_fr_10s/EVS Visit/7394377634358283409_s_4.mp4
13/1106	../../downloads/fr_10s/tes

In [10]:
test_gen = data_generator(test_paths, test_labels, batch_size=16)

In [11]:
sample_batch, sample_labels = next(test_gen)
print(f"Batch shape: {sample_batch.shape}")  # Expected: (16, 10, 224, 224, 3)
print(f"Labels shape: {sample_labels.shape}")  # Expected: (16,)

Batch shape: (16, 10, 224, 224, 3)
Labels shape: (16,)


In [19]:
X_test = np.array(X_test)
print(X_test.shape)

KeyboardInterrupt: 

In [14]:
predictions = model.predict(X_test)
predicted_classes = np.argmax(predictions, axis=1)

TypeError: Exception encountered when calling I3DModelLayer.call().

[1m'NoneType' object is not callable[0m

Arguments received by I3DModelLayer.call():
  • inputs=tf.Tensor(shape=(32, 10, 224, 224, 3), dtype=float32)

In [26]:
# Custom layer to wrap the I3D model
class I3DModelLayer(tf.keras.layers.Layer):
    def __init__(self, i3d_model, **kwargs):
        super(I3DModelLayer, self).__init__(**kwargs)
        self.i3d_model = i3d_model

    def build(self, input_shape):
        if self.i3d_model is None:
            raise ValueError("I3D model is not loaded correctly!")
        super(I3DModelLayer, self).build(input_shape)

    def call(self, inputs):
        if self.i3d_model is None:
            raise ValueError("I3D model is None during call!")
        output = self.i3d_model(rgb_input=inputs)['default']
        return output

    def get_config(self):
        config = super(I3DModelLayer, self).get_config()
        config.update({"i3d_model": None})  # Placeholder for serialization
        return config


In [30]:
import tensorflow_hub as hub

def load_i3d_model():
    i3d_model = hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']
    if i3d_model is None:
        raise ValueError("Failed to load the I3D model from TensorFlow Hub.")
    return i3d_model

# Load the model
i3d_model = load_i3d_model()

# Define the custom layer
i3d_layer = I3DModelLayer(i3d_model)


In [31]:
print(i3d_model)

ConcreteFunction Input Parameters:
Output Type:
  Dict[['default', TensorSpec(shape=(None, 400), dtype=tf.float32, name=None)]]
Captures:
  2104856196304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2124105120464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2124105121040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2124105121424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2124105121232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2124105120848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031456848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031457040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031457232: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031457424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031457616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031457808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2125031458000: TensorSpec(shape=(), dtype=tf

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

In [34]:
# Load the I3D model from TensorFlow Hub
def load_i3d_model():
    return hub.load("https://tfhub.dev/deepmind/i3d-kinetics-400/1").signatures['default']

class I3DModelLayer(tf.keras.layers.Layer):
    def __init__(self, i3d_model=None, **kwargs):
        super(I3DModelLayer, self).__init__(**kwargs)
        self.i3d_model = i3d_model

    def build(self, input_shape):
        if self.i3d_model is None:
            self.i3d_model = load_i3d_model()  # Load the I3D model if not already set
        super(I3DModelLayer, self).build(input_shape)

    def call(self, inputs):
        if self.i3d_model is None:
            raise ValueError("I3D model is None during call!")
        output = self.i3d_model(rgb_input=inputs)['default']
        return output

    def get_config(self):
        config = super(I3DModelLayer, self).get_config()
        config.update({"i3d_model": None})  # i3d_model is not serializable
        return config

    @classmethod
    def from_config(cls, config):
        # Initialize without i3d_model, then set it explicitly
        layer = cls(**config)  # Pass the config without i3d_model
        layer.i3d_model = load_i3d_model()  # Load and set the I3D model
        return layer


In [35]:
model_path = './models/2024-11-21-03-45-07-i3d-transfer-model.keras'
model = tf.keras.models.load_model(
    model_path,
    compile=False,
    custom_objects={"I3DModelLayer": I3DModelLayer}
)

In [88]:
import random
import numpy as np

def load_test_dataset(test_dir, max_videos_per_class=None):
    test_paths, test_labels, test_activities = load_dataset(test_dir)
    
    # Group paths by class
    class_dict = {}
    for path, label in zip(test_paths, test_labels):
        if label not in class_dict:
            class_dict[label] = []
        class_dict[label].append(path)

    # Now sample from each class or take the entire class if no limit is set
    limited_paths = []
    limited_labels = []
    limited_activities = []  # To hold the activities for each selected video
    for label, paths in class_dict.items():
        if max_videos_per_class is None:  # No limit, take all videos
            sampled_paths = paths
        else:  # Limit to 'max_videos_per_class' per class
            sampled_paths = random.sample(paths, min(len(paths), max_videos_per_class))
        
        limited_paths.extend(sampled_paths)
        limited_labels.extend([label] * len(sampled_paths))
        
        # Collect corresponding activities for the sampled paths
        for path in sampled_paths:
            activity = test_activities[test_paths.index(path)]  # Find the corresponding activity
            limited_activities.append(activity)
    
    return limited_paths, limited_labels, limited_activities

In [ ]:
# Load a limited set of videos
test_dir = f"../../downloads/fr_10s/test_fr_10s"
limited_paths, limited_labels, limited_activities = load_test_dataset(test_dir, max_videos_per_class=10)

# Generate the dataset (note that this should be handled in batches for efficiency)
X_test, y_test = make_dataset(limited_paths, limited_labels)
y_test = to_categorical(y_test, num_classes=num_classes)

In [89]:
# Load all videos
test_dir = f"../../downloads/fr_10s/test_fr_10s"
limited_paths, limited_labels, limited_activities = load_test_dataset(test_dir)

# Generate the dataset (note that this should be handled in batches for efficiency)
X_test, y_test = make_dataset(limited_paths, limited_labels)
y_test = to_categorical(y_test, num_classes=num_classes)

1/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393895605883686033_s_3.mp4
2/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393895605883686033_s_4.mp4
3/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393895605883686033_s_5.mp4
4/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393895605883686033_s_6.mp4
5/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393896533596621969_s_1.mp4
6/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393896744050019473_s_1.mp4
7/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393896744050019473_s_2.mp4
8/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393896744050019473_s_3.mp4
9/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393903942415207569_s_1.mp4
10/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\7393904316077362321_s_1.mp4
11/1106	../../downloads/fr_10s/test_fr_10s\Asleep-Trying to sleep\739

In [90]:
# Create the generator to handle batching
test_gen = data_generator(limited_paths, limited_labels, batch_size=16)

# Example to check the batch shape
sample_batch, sample_labels = next(test_gen)
print(f"Batch shape: {sample_batch.shape}")  # Expected: (16, 10, 224, 224, 3)
print(f"Labels shape: {sample_labels.shape}")  # Expected: (16,)

Batch shape: (16, 10, 224, 224, 3)
Labels shape: (16,)


In [91]:
predictions = model.predict(test_gen, steps=len(limited_paths) // 16 + 1)
predictions = predictions[:len(limited_paths)]

70/70 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step


In [ ]:
# Save predictions
predicted_classes = np.argmax(predictions, axis=1)
predicted_class_names = [class_names[i] for i in predicted_classes]

true_class_names = limited_activities

# Create a DataFrame 
df = pd.DataFrame({'Predicted Class (Numerical)': predicted_classes,
                   'Predicted Class (Name)': predicted_class_names,
                   'True Class (Numerical)': limited_labels,
                   'True Class (Name)': true_class_names
                   })

# Save CSV
date_time_format = '%Y-%m-%d-%H-%M-%S'
current_date_time_dt = dt.datetime.now()
current_date_time_string = dt.datetime.strftime(current_date_time_dt, date_time_format)

file_name = f'{current_date_time_string}-i3d-transfer-predictions.csv'
df.to_csv(file_name, index=False)

print(f"Saved {file_name}")


Predictions saved to 'predictions.csv'


In [86]:
true_class_names[2]


'Asleep-Trying to sleep'